In [ ]:
%run training_theme_classification

In [46]:
import torch 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [47]:
import pandas as pd
df = pd.read_csv('Data/actualite.csv')

In [48]:
score = pd.read_csv('Annotations/scores.csv')

In [49]:
score.shape

(87, 2)

In [50]:
df_merged = df.merge(score, on="identifiant", how="left")

In [51]:
df_merged = df_merged[~df_merged["score"].isna()]

In [52]:
df_merged

,Unnamed: 0,Unnamed: 0_x,identifiant,journal_clean,titre,annee,mois,jour,texte,keywords,Unnamed: 0_y,theme,texte_total,theme_pred_encoded,theme_pred,confidence,true_pred,theme_final,score
79,79,391,beb54101bece711668d64dfe5176bc2f38444b30eb9c31...,Le Figaro,Suspect en garde à vue,1997,8,25,l'époux d'une femme tuée de 18 coups de coutea...,NaN,209.0,actualité,Suspect en garde à vue l'époux d'une femme tué...,0.0,actualité,0.868749,True,actualité,0.250000
94,94,432,efee09cde26fba5704002a3180ab0bf7f2f711dbff957d...,Le Figaro,Rennes : nouveau meurtre,1998,1,19,"« ici, samedi 6 heures, une femme de 38 ans as...","violence, meurtre, rennes, femme, samedi",0.0,actualité,"Rennes : nouveau meurtre « ici, samedi 6 heure...",0.0,actualité,0.798279,True,actualité,0.000000
95,95,433,d183794139b099c8c366eb2482b740f413f22d62bb7d6d...,Le Figaro,Une femme médecin assassinée,1998,1,21,"- une femme de 60 ans, médecin allergologue, a...",NaN,1.0,actualité,Une femme médecin assassinée - une femme de 60...,0.0,actualité,0.857006,True,actualité,-0.250000
96,96,434,00627f5991ec8312f034b90a05650e755e28a6a8109170...,Le Figaro,Le fils du médecin interpellé,1998,1,22,"- le fils d'une femme médecin, assassinée chez...",fils,2.0,actualité,Le fils du médecin interpellé - le fils d'une ...,0.0,actualité,0.852238,True,actualité,-0.375000
103,103,456,76c61c4ffa49ea649bac5d7833787d404c3708d9773c54...,Le Figaro,Le tragique enlèvement de Caroline,1998,3,18,« une affaire sordide » : c'est le seul commen...,"bois, soir, poursuivent, investigations, inspe...",9.0,actualité,Le tragique enlèvement de Caroline « une affai...,3.0,politique,0.573615,True,actualité,-0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10490,10488,69726,bdf25683403919f60ef342ee26077f54820a3e712cd475...,Le Point,Vénissieux : un homme se filme en train de tue...,2024,2,17,le corps d'une jeune femme a été retrouvé dans...,"jeune, tuer, corps, retrouvé, filme, suicider,...",251.0,actualité,Vénissieux : un homme se filme en train de tue...,0.0,actualité,0.864363,True,actualité,-0.125000
10569,10567,70473,5dee4cabc6efe7a314ca832711d82445672d002596934d...,Le Figaro,Meurtre conjugal près de Nice : le suspect mis...,2024,2,29,"le figaro niceaprès 48 heures de garde à vue, ...","mis, suspect, examen",352.0,actualité,Meurtre conjugal près de Nice : le suspect mis...,0.0,actualité,0.832457,True,actualité,0.750000
10775,10773,72137,e47da7a5c284da5d8114c1014ee551115b22c5446a8e6c...,Libération,Le réalisateur Nils Tavernier visé par deux pl...,2024,4,5,jennifer covillault miramont et laura lardeux ...,"obs, réalisateur, nils, jennifer, miramont, fa...",387.0,actualité,Le réalisateur Nils Tavernier visé par deux pl...,0.0,actualité,0.593339,True,actualité,0.500000
10797,10795,72299,997400ff56c84208c33e328a30abf85f74b90b963b3b33...,Le Figaro,Football: Florencia Guiñazu tuée par son ex-co...,2024,4,9,un féminicide a touché ce samedi le monde du f...,"domicile, football:, joueuse, guiñazu, florencia",384.0,actualité,Football: Florencia Guiñazu tuée par son ex-co...,0.0,actualité,0.790045,True,actualité,0.428571


In [53]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(
    df_merged[["texte_total", "score"]].dropna(),
    test_size=0.2,
    random_state=42
)


train_df = train_df.reset_index(drop=True)
test_df = test_df.reset_index(drop=True)


In [66]:
from transformers import CamembertTokenizer

tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
train_dataset = RegressionDataset(train_df, tokenizer)
train_dataset = RegressionDataset(train_df, tokenizer, inference=False)
full_df = df_merged[["texte_total"]].dropna().reset_index(drop=True)
full_dataset = RegressionDataset(full_df, tokenizer, inference=True)
full_loader = DataLoader(full_dataset, batch_size=16)

In [55]:
import torch.nn as nn
from transformers import CamembertModel

class CamembertRegressor(nn.Module):
    def __init__(self):
        super().__init__()
        self.backbone = CamembertModel.from_pretrained("camembert-base")

        # Freeze all CamemBERT layers
        for param in self.backbone.parameters():
            param.requires_grad = False

        # Use [CLS] token (token 0) → Linear → scalar output
        self.regressor = nn.Linear(768, 1)

    def forward(self, input_ids, attention_mask):
        with torch.no_grad():
            outputs = self.backbone(input_ids=input_ids, attention_mask=attention_mask)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [batch, 768]
        score = self.regressor(cls_embedding).squeeze(1)    # [batch]
        return score


In [63]:
from torch.utils.data import Dataset

class RegressionDataset(Dataset):
    def __init__(self, df, tokenizer, max_length=256, inference=False):
        self.df = df.reset_index(drop=True)
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.inference = inference

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = str(self.df.iloc[idx]["texte_total"])
        inputs = self.tokenizer(
            text,
            padding="max_length",
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt"
        )
        item = {
            "input_ids": inputs["input_ids"].squeeze(0),
            "attention_mask": inputs["attention_mask"].squeeze(0),
        }

        if not self.inference:
            score = self.df.iloc[idx]["score"]
            item["score"] = torch.tensor(score, dtype=torch.float)

        return item


In [57]:
model = CamembertRegressor().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.regressor.parameters(), lr=1e-4)


In [58]:
from torch.utils.data import DataLoader

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

In [59]:
from sklearn.metrics import mean_squared_error, r2_score
from tqdm import tqdm

num_epochs = 30

for epoch in tqdm(range(num_epochs)):
    model.train()
    total_loss = 0

    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['score'].to(device)  
        outputs = model(input_ids, attention_mask) 
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    # Évaluation
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['score'].to(device)

            outputs = model(input_ids, attention_mask)

            all_preds.extend(outputs.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    mse = mean_squared_error(all_labels, all_preds)
    r2 = r2_score(all_labels, all_preds)

    print(f"Epoch {epoch+1}/{num_epochs} — Loss: {total_loss/len(train_loader):.4f} | "
          f"MSE: {mse:.4f} | R²: {r2:.4f}")



  3%|▎         | 1/30 [00:01<00:44,  1.53s/it]

Epoch 1/30 — Loss: 0.2193 | MSE: 0.1678 | R²: -0.0709


  7%|▋         | 2/30 [00:03<00:42,  1.53s/it]

Epoch 2/30 — Loss: 0.2065 | MSE: 0.1638 | R²: -0.0456


 10%|█         | 3/30 [00:04<00:40,  1.50s/it]

...

Epoch 28/30 — Loss: 0.1883 | MSE: 0.1555 | R²: 0.0076


 97%|█████████▋| 29/30 [00:42<00:01,  1.47s/it]

Epoch 29/30 — Loss: 0.1963 | MSE: 0.1556 | R²: 0.0072


100%|██████████| 30/30 [00:44<00:00,  1.47s/it]

Epoch 30/30 — Loss: 0.2064 | MSE: 0.1556 | R²: 0.0068


In [60]:
full_df = df_merged[["texte_total"]].dropna().reset_index(drop=True)
full_dataset = RegressionDataset(full_df, tokenizer)

In [61]:
full_loader = DataLoader(full_dataset, batch_size=16)

In [67]:
model.eval()
all_preds = []

with torch.no_grad():
    for batch in full_loader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        outputs = model(input_ids, attention_mask)
        all_preds.extend(outputs.cpu().numpy())

df_merged = df_merged.dropna(subset=["texte_total"]).reset_index(drop=True)
df_merged["score_pred"] = all_preds


In [68]:
df_merged = df_merged.dropna(subset=["texte_total"]).reset_index(drop=True)
df_merged["score_pred"] = all_preds

In [69]:
df_merged

,Unnamed: 0,Unnamed: 0_x,identifiant,journal_clean,titre,annee,mois,jour,texte,keywords,Unnamed: 0_y,theme,texte_total,theme_pred_encoded,theme_pred,confidence,true_pred,theme_final,score,score_pred
0,79,391,beb54101bece711668d64dfe5176bc2f38444b30eb9c31...,Le Figaro,Suspect en garde à vue,1997,8,25,l'époux d'une femme tuée de 18 coups de coutea...,NaN,209.0,actualité,Suspect en garde à vue l'époux d'une femme tué...,0.0,actualité,0.868749,True,actualité,0.250000,0.025689
1,94,432,efee09cde26fba5704002a3180ab0bf7f2f711dbff957d...,Le Figaro,Rennes : nouveau meurtre,1998,1,19,"« ici, samedi 6 heures, une femme de 38 ans as...","violence, meurtre, rennes, femme, samedi",0.0,actualité,"Rennes : nouveau meurtre « ici, samedi 6 heure...",0.0,actualité,0.798279,True,actualité,0.000000,0.037271
2,95,433,d183794139b099c8c366eb2482b740f413f22d62bb7d6d...,Le Figaro,Une femme médecin assassinée,1998,1,21,"- une femme de 60 ans, médecin allergologue, a...",NaN,1.0,actualité,Une femme médecin assassinée - une femme de 60...,0.0,actualité,0.857006,True,actualité,-0.250000,0.034296
3,96,434,00627f5991ec8312f034b90a05650e755e28a6a8109170...,Le Figaro,Le fils du médecin interpellé,1998,1,22,"- le fils d'une femme médecin, assassinée chez...",fils,2.0,actualité,Le fils du médecin interpellé - le fils d'une ...,0.0,actualité,0.852238,True,actualité,-0.375000,0.046982
4,103,456,76c61c4ffa49ea649bac5d7833787d404c3708d9773c54...,Le Figaro,Le tragique enlèvement de Caroline,1998,3,18,« une affaire sordide » : c'est le seul commen...,"bois, soir, poursuivent, investigations, inspe...",9.0,actualité,Le tragique enlèvement de Caroline « une affai...,3.0,politique,0.573615,True,actualité,-0.500000,0.006438
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84,10488,69726,bdf25683403919f60ef342ee26077f54820a3e712cd475...,Le Point,Vénissieux : un homme se filme en train de tue...,2024,2,17,le corps d'une jeune femme a été retrouvé dans...,"jeune, tuer, corps, retrouvé, filme, suicider,...",251.0,actualité,Vénissieux : un homme se filme en train de tue...,0.0,actualité,0.864363,True,actualité,-0.125000,0.044556
85,10567,70473,5dee4cabc6efe7a314ca832711d82445672d002596934d...,Le Figaro,Meurtre conjugal près de Nice : le suspect mis...,2024,2,29,"le figaro niceaprès 48 heures de garde à vue, ...","mis, suspect, examen",352.0,actualité,Meurtre conjugal près de Nice : le suspect mis...,0.0,actualité,0.832457,True,actualité,0.750000,0.024208
86,10773,72137,e47da7a5c284da5d8114c1014ee551115b22c5446a8e6c...,Libération,Le réalisateur Nils Tavernier visé par deux pl...,2024,4,5,jennifer covillault miramont et laura lardeux ...,"obs, réalisateur, nils, jennifer, miramont, fa...",387.0,actualité,Le réalisateur Nils Tavernier visé par deux pl...,0.0,actualité,0.593339,True,actualité,0.500000,0.045055
87,10795,72299,997400ff56c84208c33e328a30abf85f74b90b963b3b33...,Le Figaro,Football: Florencia Guiñazu tuée par son ex-co...,2024,4,9,un féminicide a touché ce samedi le monde du f...,"domicile, football:, joueuse, guiñazu, florencia",384.0,actualité,Football: Florencia Guiñazu tuée par son ex-co...,0.0,actualité,0.790045,True,actualité,0.428571,0.061558
